In [1]:
#Pkg.add("Combinatorics")
#Pkg.add("Pipe")
#Pkg.add("Lazy")

In [2]:
using Combinatorics
using Lazy: map, filter, any, all, @as, @>, @>>
using Base.Iterators: flatten, product

In [3]:
abstract type Node end

In [4]:
mutable struct SudokuNode <: Node
    coordinates::Tuple{Int,Int}
    cell::Int
    value::Int
    
    possible_values::Vector{Int}
    
    function Node(coordinates::Tuple{Int,Int})
        return new(coordinates,0,0,[])
    end
end

In [5]:
function get_coordinates(node::SudokuNode)::Tuple{Int,Int,Int}
    return (node.coordinates..., node.cell)
end

get_coordinates (generic function with 1 method)

In [6]:
function Base.isequal(a::SudokuNode,b::SudokuNode)::Bool
    a,b = get_coordinates.((a,b))
    return all(a.==b)
end

In [7]:
function set_cell!(node::SudokuNode, size::Int)::SudokuNode
    row, col = node.coordinates .- 1
    node.cell = 1 + floor(((col//size) + row) - (row%size))
    return node
end

set_cell! (generic function with 1 method)

In [8]:
function get_value(node::SudokuNode)::Int
    return node.value
end

get_value (generic function with 1 method)

In [9]:
struct Edge
    nodes::Tuple{SudokuNode,SudokuNode}
    
    function Edge(nodes::Vector{<:Node})
        return new(tuple(nodes...))
    end
end

In [10]:
function get_nodes(edge::Edge)::Vector{SudokuNode}
    return collect(edge.nodes)
end

function get_nodes(edges::Array{Edge})::Vector{SudokuNode}
    return get_nodes.(edges)
end

get_nodes (generic function with 2 methods)

In [11]:
function are_adjacent(edge::Edge)::Bool
    a,b = get_coordinates.(edge.nodes)
    return any(a .== b)
end

are_adjacent (generic function with 1 method)

In [12]:
function Base.in(node::SudokuNode, edge::Edge)::Bool
    return any(isequal.(fill(node), edge.nodes))
end

In [13]:
function nodes_to_edges(nodes::Vector{SudokuNode})::Vector{Edge}
    return @>> begin
        combinations(nodes,2) 
        map(Edge) 
        filter(are_adjacent)
    end
end

nodes_to_edges (generic function with 1 method)

In [14]:
abstract type Graph end
abstract type SimpleGraph <: Graph end

In [15]:
mutable struct SudokuGraph <: SimpleGraph
    nodes::Vector{SudokuNode}
    edges::Vector{Edge}
    
    size::Int

    function SudokuGraph(size::Int)        
        #=
        The node coordinates are (row, column, cell),
        generated by creating the cartesian set of (row, col)
        and then assigning the cell based on a formula.
        =#
        
        nodes = @as x begin
            1:size^2
            product(x,x) 
            map(Node, x)
            set_cell!.(x, size) 
            reshape(x, size^4)
        end
        
        edges = nodes_to_edges(nodes)
                
        return new(nodes, edges, size)
    end
end

In [16]:
function get_node(coordinates::Tuple{Int,Int}, graph::SudokuGraph)::SudokuNode
    return @>> begin 
        graph.nodes 
        filter(node -> node.coordinates == coordinates) 
        pop!
    end
end

get_node (generic function with 1 method)

In [17]:
function get_neighbors(node::SudokuNode, graph::SudokuGraph)::Vector{SudokuNode}
    return @>> graph.edges begin
        filter(edge -> node in edge)
        map(get_nodes)
        flatten
        collect
        filter(x -> !isequal(node,x))
    end
end

get_neighbors (generic function with 1 method)

In [18]:
function get_saturated_values(node::SudokuNode, graph::SudokuGraph)::Vector{Int}
    return @>> begin
        get_neighbors(node, graph)
        filter(x -> x.value > 0)
        get_value.()
        unique
    end
end

get_saturated_values (generic function with 1 method)

In [19]:
function get_saturation(node::SudokuNode, graph::SudokuGraph)::Int
    return get_saturated_values(node, graph) |> length
end

get_saturation (generic function with 1 method)

In [20]:
function get_possible_values(node::SudokuNode, graph::SudokuGraph)::Vector{Int}
    sv = get_saturated_values(node, graph)
    return collect(filter(x->x ∉ sv, 1:graph.size^2))  
end

get_possible_values (generic function with 1 method)

In [21]:
function set_possible_values!(node::SudokuNode, graph::SudokuGraph)::SudokuNode
    node.possible_values = get_possible_values(node, graph)
    return node
end

set_possible_values! (generic function with 1 method)

In [22]:
function eliminate_possibility!(node::SudokuNode, value::Int)::SudokuNode
    if value in node.possible_values
        pop!(node.possible_values, value)
    end
    return node
end

eliminate_possibility! (generic function with 1 method)

In [23]:
function set_value!(node::SudokuNode, value::Int)::Node
    node.value = value
    return node
end


set_value! (generic function with 1 method)

In [24]:
@timev s = SudokuGraph(3);

  0.000612 seconds (9.97 k allocations: 625.781 KiB)
elapsed time (ns): 611600
bytes allocated:   640800
pool allocs:       9972
malloc() calls:    2
realloc() calls:   1


In [25]:
length(s.edges)

810

In [26]:
length(s.nodes)

81

In [27]:
a,b = get_node.(((1,1),(1,2)), fill(s))

(SudokuNode((1, 1), 1, 0, Int64[]), SudokuNode((1, 2), 1, 0, Int64[]))

In [28]:
@time get_neighbors(a, s)

  0.000038 seconds (839 allocations: 92.391 KiB)


20-element Array{SudokuNode,1}:
 SudokuNode((2, 1), 1, 0, Int64[])
 SudokuNode((3, 1), 1, 0, Int64[])
 SudokuNode((4, 1), 4, 0, Int64[])
 SudokuNode((5, 1), 4, 0, Int64[])
 SudokuNode((6, 1), 4, 0, Int64[])
 SudokuNode((7, 1), 7, 0, Int64[])
 SudokuNode((8, 1), 7, 0, Int64[])
 SudokuNode((9, 1), 7, 0, Int64[])
 SudokuNode((1, 2), 1, 0, Int64[])
 SudokuNode((2, 2), 1, 0, Int64[])
 SudokuNode((3, 2), 1, 0, Int64[])
 SudokuNode((1, 3), 1, 0, Int64[])
 SudokuNode((2, 3), 1, 0, Int64[])
 SudokuNode((3, 3), 1, 0, Int64[])
 SudokuNode((1, 4), 2, 0, Int64[])
 SudokuNode((1, 5), 2, 0, Int64[])
 SudokuNode((1, 6), 2, 0, Int64[])
 SudokuNode((1, 7), 3, 0, Int64[])
 SudokuNode((1, 8), 3, 0, Int64[])
 SudokuNode((1, 9), 3, 0, Int64[])

In [29]:
@time get_saturation(a, s)

  0.000057 seconds (846 allocations: 93.250 KiB)


0

In [30]:
set_value!(b, 3)

SudokuNode((1, 2), 1, 3, Int64[])

In [31]:
@time get_saturated_values(a, s)

  0.000135 seconds (847 allocations: 93.312 KiB)


1-element Array{Int64,1}:
 3

In [32]:
@show a;

a = SudokuNode((1, 1), 1, 0, Int64[])


In [33]:
@timev set_possible_values!(a, s)

  0.000047 seconds (850 allocations: 93.750 KiB)
elapsed time (ns): 47400
bytes allocated:   96000
pool allocs:       849
non-pool GC allocs:1


SudokuNode((1, 1), 1, 0, [1, 2, 4, 5, 6, 7, 8, 9])

In [34]:
@show a;

a = SudokuNode((1, 1), 1, 0, [1, 2, 4, 5, 6, 7, 8, 9])
